# From EMA Selection to Intervals Data

* Enter an EMA expression for a selected passage of any piece and return various CRIM intervals analyses of those notes.
* Begin by opening a score on the CRIM New Relationships form:  https://crimproject.org/relationships/new/
* You don't need to create a new Relationship, but simply select the notes you want.
* Copy the EMA address (which you will need below)


### A.  Import CRIM Intervals

In [6]:
import intervals
from intervals import ema2ex
from intervals import importScore 
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [7]:
piece = importScore('https://crimproject.org/mei/CRIM_Model_0046.mei')

# piece = importScore('https://raw.githubusercontent.com/RichardFreedman/CRIM_additional_works/main/MEI/Palestrina_MIOM_1.musicxml')

Previously imported piece detected.


### B. Search for Patterns with Score Selection and ema2ex

* Begin by opening a score on the CRIM New Relationships form:  https://crimproject.org/relationships/new/
* You don't need to create a new Relationship, but simply **select the notes you want**.
* Copy the ema selection, such as: `1-6/1-2,1-2,1-2,1-2,1-2,2/@1-3+@all,@1-3+@all,@1-3+@1-3,@1+@1-3,@all+@1-3,@1`
* Make sure you import correct **score** of the piece into this Notebook
* Paste the **`EMA address`** (the measures/staves/beats only) into the **`ema_pattern`** box below
* Select the interval type, ngram length, 

* Check **`kind`** for `"d"` or `"c"` as needed
* Select

In [8]:
@interact
def get_ema_intervals(ema_pattern="", 
                      kind=["d", "q", "c", "z"], 
                      length=[1, 2, 3, 4, 5, 6], 
                     display=['melodic', 'harmonic', 'modules']):
    
    ema_clean = ema_pattern.replace('@all', '@1-4')
    nr = piece.notes()
    detNr = piece.detailIndex(nr, offset=True)
    mel = piece.melodic(kind=kind)
    har = piece.harmonic(kind=kind)
    excerpt = ema2ex(ema_clean, detNr)
    excerptOffsets = excerpt.index.get_level_values(2)
    
    if display == 'melodic':
        ngrams = piece.ngrams(df=mel, n=length)
        return ngrams.reindex(index = excerptOffsets).fillna('-')
    elif display == 'harmonic':
        ngrams = piece.ngrams(df=har, n=length)
        return ngrams.reindex(index = excerptOffsets).fillna('-')
    elif display == 'modules':
        ngrams = piece.ngrams(df=har, other=mel, n=length)
        return ngrams.reindex(index = excerptOffsets).fillna('-')

interactive(children=(Text(value='', description='ema_pattern'), Dropdown(description='kind', options=('d', 'q…

In [9]:
ema_pattern = "1/5-6/@1-3+@1-3"
kind = "d"

In [10]:
def convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

In [11]:
ema_clean = ema_pattern.replace('@all', '@1-4')
nr = piece.notes()
detNr = piece.detailIndex(nr, offset=True)
detNr = detNr.applymap(convertTuple)
mel = piece.melodic(kind=kind)
har = piece.harmonic(kind=kind)
excerpt = ema2ex(ema_clean, detNr)
# excerptOffsets = excerpt.index.get_level_values(2)

IndexError: index 4 is out of bounds for axis 0 with size 4

In [12]:
string = "1, 2-4"

In [13]:
measures = []
chunks = string.split(',')
for chunk in chunks:
    spans = chunk.split('+')
    for span in spans:
        if len(span) == 1:
            print(span)
        else:
            ends = tuple(int(m) for m in span.split('-'))
            for m in range(ends[0], ends[1] + 1):
                print(m)

1
2
3
4


In [14]:
measures = []
chunks = string.split(',')
for chunk in chunks:
    spans = chunk.split('+')
    for span in spans:
        if len(span) == 1:
            yield span
        else:
            ends = tuple(int(m) for m in span.split('-'))
            for m in range(ends[0], ends[1] + 1):
                yield m

SyntaxError: 'yield' outside function (1001506269.py, line 7)